# CH 03
## 데이터 준비
본 챕터에서 사용하는 데이터는 Audioscrobbler에서 공개한 데이터셋을 사용합니다. 2005년에 last.fm에서 배포한 데이터셋으로 사용자들에게 음악을 추천하기 위해서 사용된 정보라고 합니다. 필요한 파일들을 압축 해제할 수 있도록 아래 명령어를 실행합니다.

In [6]:
!curl -L -o dataset.tar.gz http://www.iro.umontreal.ca/~lisa/datasets/profiledata_06-May-2005.tar.gz
!tar -xzf dataset.tar.gz
!rm dataset.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  129M  100  129M    0     0  3230k      0  0:00:41  0:00:41 --:--:-- 3141k23 30.3M    0     0  2735k      0  0:00:48  0:00:11  0:00:37 4463k


In [1]:
import findspark
findspark.find()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Spark") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

sc = spark.sparkContext

from pyspark.sql import functions as f
from pyspark.sql import Window

교차 최소 제곱 추천 알고리즘

In [44]:
user_artist = spark.read.option('inferSchema', True).option('sep', ' ').csv('profiledata_06-May-2005/user_artist_data.txt').toDF('user', 'artist', 'runtime')

In [45]:
user_artist

user,artist,runtime
1000002,1,55
1000002,1000006,33
1000002,1000007,8
1000002,1000009,144
1000002,1000010,314
1000002,1000013,8
1000002,1000014,42
1000002,1000017,69
1000002,1000024,329
1000002,1000025,1


In [38]:
user_artist.agg(f.min('user'), f.max('user'), f.min('artist'), f.max('artist'))

min(user),max(user),min(artist),max(artist)
1000002,9875,1,9999998


In [41]:
artist_by_id = spark.read.option('inferSchema', True).option('sep', '\t').csv('profiledata_06-May-2005/artist_data.txt').toDF('id', 'name')
artist_by_id.count()

1848671

In [47]:
artist_alias = dict(spark.read.option('inferSchema', True).option('sep', '\t').csv('profiledata_06-May-2005/artist_alias.txt').collect())
artist_alias[1208690]

1003926

In [43]:
artist_by_id.filter(f.col('id').isin(1208690, 1003926))

id,name
1208690,Collective Souls
1003926,Collective Soul


In [ ]:
def buildCounts